# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,carnarvon,-24.8667,113.6333,31.33,32,21,2.39,AU,1712547376
1,1,adamstown,-25.0660,-130.1015,23.89,87,77,4.15,PN,1712547312
2,2,edinburgh of the seven seas,-37.0676,-12.3116,18.45,95,100,7.26,SH,1712547153
3,3,petrozavodsk,61.7849,34.3469,5.82,100,100,8.94,RU,1712547868
4,4,taman',45.2117,36.7161,10.80,66,1,7.47,RU,1712547868


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
import holoviews as hv
from holoviews.element.tiles import OSM

# Configure the map plot
map_humidity = city_data_df.hvplot.points(
    "Lng",             # Adjust column name for longitude if necessary
    "Lat",             # Adjust column name for latitude if necessary
    geo=True,
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    size="Humidity",   # Size of the point represents humidity
    color="City",      # Use 'City' column for colors
    cmap='Category10', # Choose a colormap with distinct colors for categories
    alpha=0.7          # Set transparency to 0.7
).opts(
    xlabel="Longitude",
    ylabel="Latitude"
)

# Display the map plot
map_humidity



c:\Users\Sarah Son Kim\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Filter cities based on criteria (e.g., temperature between 20°C and 25°C, humidity below 50%, cloudiness below 30%, wind speed below 5 m/s)
filtered_cities_df = city_data_df[(city_data_df['Max Temp'] >= 20) & (city_data_df['Max Temp'] <= 25) & 
                                  (city_data_df['Humidity'] < 50) & (city_data_df['Cloudiness'] < 50) & 
                                  (city_data_df['Wind Speed'] < 5)].copy()

# Drop any rows with null values
filtered_cities_df.dropna(inplace=True)

# Display sample data
filtered_cities_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
75,75,jiexiu,37.0244,111.9125,21.86,29,43,2.11,CN,1712547353
129,129,atar,20.5169,-13.0499,21.75,38,0,4.57,MR,1712547882
195,195,timimoun,29.2639,0.2310,23.21,11,0,3.09,DZ,1712547365
208,208,margaret river,-33.9500,115.0667,22.89,38,3,3.99,AU,1712547324
358,358,broken hill,-31.9500,141.4333,21.10,37,9,4.29,AU,1712547270


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()


,City,Country,Lat,Lng,Humidity,Hotel Name
75,jiexiu,CN,37.0244,111.9125,29,
129,atar,MR,20.5169,-13.0499,38,
195,timimoun,DZ,29.2639,0.2310,11,
208,margaret river,AU,-33.9500,115.0667,38,
358,broken hill,AU,-31.9500,141.4333,37,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000  # Define the search radius in meters
params = {
    "type": "hotel",
    "radius": radius,
    "limit": 1,  # Limit to only 1 result per city
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{radius}@{lat},{lng}"
    params["bias"] = f"point:{lat},{lng}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
hotel_df.head()



Starting hotel search
jiexiu - nearest hotel: No hotel found
atar - nearest hotel: No hotel found
timimoun - nearest hotel: No hotel found
margaret river - nearest hotel: No hotel found
broken hill - nearest hotel: No hotel found
al fuwayliq - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
75,jiexiu,CN,37.0244,111.9125,29,No hotel found
129,atar,MR,20.5169,-13.0499,38,No hotel found
195,timimoun,DZ,29.2639,0.2310,11,No hotel found
208,margaret river,AU,-33.9500,115.0667,38,No hotel found
358,broken hill,AU,-31.9500,141.4333,37,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
# Configure the map plot with hotel names and country information in the hover message
map_with_hotel_info = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    size="Humidity",
    hover_cols=["City", "Country", "Hotel Name"],  # Add hotel name and country to hover message
    color="City",
    cmap='Category10', # Choose a colormap with distinct colors for categories
    alpha=0.7
).opts(
    xlabel="Longitude",
    ylabel="Latitude"
)

# Display the map plot
map_with_hotel_info


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)